## Import cities_df

In [25]:
import pandas as pd

In [26]:
cities_df= pd.read_pickle("../01_cities/out_dfs/cities_2.pkl")  

In [27]:
population_df= pd.read_pickle("../01_cities/out_dfs/population_2.pkl")  

In [28]:
population_df

,city_id,pop_des,pop_value
0,0,Population (2021 except where stated),8799800
1,1,Population (Jan. 2019),2165423
2,2,Population (2021),3677472
3,3,Population (2021 Census),13010112
4,4,Population (31 December 2021),15840900
5,5,Population (2018),1620343
6,6,Population,5351935
7,7,Population (2018),3223334
8,8,Population (1 January 2021),2962180
9,9,Population (31 December 2019),NotAvailable


In [29]:
cities_df

,city_id,city,country,country_iso,coordination,lat,long
0,0,London,England,GB,51.50722°N 0.12750°W,51.507220,-0.127500
1,1,Paris,France,FR,48.856613°N 2.352222°E,48.856613,2.352222
2,2,Berlin,Germany,DE,52.52000°N 13.40500°E,52.520000,13.405000
3,3,Moscow,Russia,RU,55.75583°N 37.61722°E,55.755830,37.617220
4,4,Istanbul,Turkey,TR,41.01361°N 28.95500°E,41.013610,28.955000
5,5,Barcelona,Spain,ES,41.383°N 2.183°E,41.383000,2.183000
6,6,Saint Petersburg,Russia,RU,59.93750°N 30.30861°E,59.937500,30.308610
7,7,Madrid,Spain,ES,40.41667°N 3.70250°W,40.416670,-3.702500
8,8,Kyiv,Ukraine,UA,50.45000°N 30.52333°E,50.450000,30.523330
9,9,Rome,Italy[a],IT,41.89333°N 12.48278°E,41.893330,12.482780


## Get Weather from API

In [24]:
import requests
import json
from api_key import API_key

In [ ]:
# coor= "52.52000°N 13.40500°E"  
# lat,long = ConvertCoord(coor)
# print(lat,long)
# url_weather= f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={long}&appid={API_key}"
# weather_response = requests.get(url_weather)
# weather_json_l= weather_response.json()
# #print(json.dumps(weather_json_l, indent=2))

52.52 13.405


In [34]:
weather_json_l=[]
for i in range(len(cities_df)):
    lat=cities_df.lat[i]
    long=cities_df.long[i]
    url_weather= f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={long}&appid={API_key}&units=metric"
    weather_response = requests.get(url_weather)
    if weather_response.status_code != 200:
        print(weather_response.text)
        weather_json = ""
    else:
        weather_json_l.append(weather_response.json())
       

In [ ]:
# from pprint import pprint
# pprint(weather_json_l[0])

In [36]:
cities_weath = {"city_id":[]
                ,"forcast_time":[]
                ,"outlook":[]
                ,"temperature":[]
                ,"feels_like":[]
                ,"wind_speed":[]
                ,"rain_prob":[]
                }

for j,city_json in enumerate(weather_json_l):
    for time in city_json["list"]:
        cities_weath["city_id"].append(j)
        cities_weath["forcast_time"].append(time["dt_txt"])
        cities_weath["temperature"].append(time["main"]["temp"])
        cities_weath["outlook"].append(time["weather"][0]["description"])
        cities_weath["feels_like"].append(time["main"]["feels_like"])
        cities_weath["wind_speed"].append(time["wind"]["speed"])
        cities_weath["rain_prob"].append(time["pop"])
        

In [37]:
cities_weather_df = pd.DataFrame(cities_weath)

## Export Final df

In [38]:
cities_weather_df

,city_id,forcast_time,outlook,temperature,feels_like,wind_speed,rain_prob
0,0,2023-01-04 15:00:00,broken clouds,13.20,12.53,7.87,0.00
1,0,2023-01-04 18:00:00,overcast clouds,11.89,11.17,7.16,0.00
2,0,2023-01-04 21:00:00,overcast clouds,10.69,9.85,6.41,0.00
3,0,2023-01-05 00:00:00,overcast clouds,11.08,10.05,6.51,0.00
4,0,2023-01-05 03:00:00,overcast clouds,10.71,9.90,4.80,0.00
...,...,...,...,...,...,...,...
515,12,2023-01-09 00:00:00,overcast clouds,3.94,3.94,0.25,0.00
516,12,2023-01-09 03:00:00,overcast clouds,3.90,3.90,0.82,0.00
517,12,2023-01-09 06:00:00,overcast clouds,4.27,4.27,1.29,0.12
518,12,2023-01-09 09:00:00,light rain,6.16,3.22,4.12,0.50


In [39]:
cities_weather_df.to_pickle("weather.pkl")